<a href="https://colab.research.google.com/github/cfarhutchins/technical_direction_service/blob/master/BOW_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install GetOldTweets3
pip install vaderSentiment

In [0]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils
import GetOldTweets3 as got
from sklearn.feature_extraction.text import CountVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import nltk 
import bs4 as BeautifulSoup
nltk.download('stopwords')
import html5lib


In [94]:
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('reparations')\
                                           .setSince('2019-07-04')\
                                           .setUntil('2019-07-10')\
                                           .setMaxTweets(50_000)
tweet = got.manager.TweetManager.getTweets(tweetCriteria)

repar_tweets = []

for i in tweet:
  repar_tweets.append(i.text)

An error occured during an HTTP request: HTTP Error 500: Internal Server Error
Try to open in browser: https://twitter.com/search?q=reparations%20since%3A2019-07-04%20until%3A2019-07-10&src=typd


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
analyzer = SentimentIntensityAnalyzer()

In [0]:
df_repar = pd.DataFrame(repar_tweets)
df_repar.rename(columns={0:'tweet'}, inplace= True)
df_repar['feels'] = df_repar['tweet'].apply(lambda i: analyzer.polarity_scores(i))
compound = []

for dct in df_repar['feels']:
  compound.append(dct['compound'])

negative = []
  
for dct in df_repar['feels']:
  negative.append(dct['neg'])
  
neutral = [] 
  
for dct in df_repar['feels']:
  neutral.append(dct['neu'])
  
positive = []

for dct in df_repar['feels']:
  positive.append(dct['pos'])
  
df_repar['compound'] = compound
df_repar['negative'] = negative
df_repar['neutral'] = neutral
df_repar['positive'] = positive

df_repar.drop(columns = 'feels', inplace = True)

In [39]:
df_repar.head()

,tweet,compound,negative,neutral,positive
0,He needs to shut up. He wants google to provid...,0.0000,0.000,1.000,0.000
1,@BarackObama was a good lesson for Black Ameri...,-0.5095,0.189,0.735,0.076
2,Where are my reparations?? https://www.globalr...,0.0000,0.000,1.000,0.000
3,But the 1 point that made the most sense which...,-0.8685,0.223,0.650,0.127
4,Let's talk reparations: https://youtu.be/qSF95...,0.0000,0.000,1.000,0.000


In [0]:
from nltk.corpus import stopwords
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.validation import column_or_1d

In [0]:
word = re.compile('[^\s]+')


In [42]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    #text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df_repar['clean_text'] = df_repar['tweet'].apply(clean_text)
#print_plot(10)
df_repar.head()

,tweet,compound,negative,neutral,positive,clean_text
0,He needs to shut up. He wants google to provid...,0.0000,0.000,1.000,0.000,needs shut wants google provide con content ti...
1,@BarackObama was a good lesson for Black Ameri...,-0.5095,0.189,0.735,0.076,barackobama good lesson black americans enemie...
2,Where are my reparations?? https://www.globalr...,0.0000,0.000,1.000,0.000,reparations https wwwglobalresearchca theirish...
3,But the 1 point that made the most sense which...,-0.8685,0.223,0.650,0.127,1 point made sense eventually led 2 reparation...
4,Let's talk reparations: https://youtu.be/qSF95...,0.0000,0.000,1.000,0.000,lets talk reparations https youtube qsf95mcgwp4


In [0]:
my_tags = ['reparations',
           'slavery',
           'descendants', 
           'stupid',
           'civil war',
           'ancestors',
           'dead',
           'hell',
           'racism',
           'abuse',
           'shit',
           'bitch',
           'bullshit',
           'vote',
           'pay',
           'pain',
           'suffering',
           'illegal aliens',
           'dead',
           'scam',
           'chattel', 
           'forced',
           'stealing',
           'end',
           'stealth',
           'good',
           'luck', 
           'happy',
           'win', 
           'grandfathers',
           'grandmothers',
           'honestly',
           'need',
           'better',
           'freedom',
           'wealth', 
           'redistribution',
           'support',
           'public',
           'forum',
           'argument',
           'peace',
           'love',
           'justice',
           'claim'
]


In [83]:
pingbit = pd.Series(my_tags)
pingbit.head()

0    reparations
1        slavery
2    descendants
3         stupid
4      civil war
dtype: object

In [0]:
cv = CountVectorizer(ngram_range=(2,2),binary=True, stop_words='english')


In [0]:
x = df_repar['tweet']
y = df_repar['compound']



In [0]:
from sklearn.model_selection import train_test_split, cross_val_score

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= .33)

X_train = cv.fit_transform(x_train)

X_test = cv.transform(x_test)

In [0]:
from sklearn.linear_model import Ridge

rdg = Ridge()

In [0]:
rdg.fit(X_train, y_train)

In [0]:
rdg.predict(X_test)

In [0]:
rdg.score(X_test, y_test)
pingwords = cv.get_feature_names()

In [0]:
df_pingwords = pd.DataFrame(rdg.coef_, index = pingwords).sort_values(by=0)


In [0]:
ping_scores = []

for data in df_pingwords[0]:
  ping_scores.append(data)

df_ping_scores = pd.DataFrame(ping_scores)

In [0]:
df_ping_scores.head()

In [0]:
pingwords_lst = []

for i in pingwords:
  pingwords_lst.append(i)

In [0]:
df_pingwords = df_pingwords.index
  

In [0]:
pingwords_lst = df_pingwords.to_list()

In [0]:
df_pingwords = pd.DataFrame(pingwords_lst)

In [0]:
df_pingwords

In [0]:
df_pingwords.rename(columns={0:'ping pair'}, inplace= True)

In [0]:
df_pingwords.head()

In [0]:
df_attention = df_pingwords.join(df_ping_scores)
df_attention.rename(columns = {0:'score'}, inplace=True)
#df_pingwords.rename(columns={0:'attention'})

In [0]:
df_attention.head()

In [0]:
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.validation import column_or_1d

In [91]:
train_size = int(len(df_repar) * .7)
train_posts = df_repar['clean_text'][:train_size]
train_tags = df_repar['compound'][:train_size]

test_posts = df_repar['clean_text'][train_size:]
test_tags = df_repar['compound'][train_size:]

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)


encoder = TolerantLabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.fit_transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

batch_size = 32
epochs = 2

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='Adagrad',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 15951 samples, validate on 1773 samples
Epoch 1/2
 3104/15951 [====>.........................] - ETA: 18s - loss: 5.9524 - acc: 0.2622

KeyboardInterrupt: ignored

In [0]:
history.on_epoch_end(epoch= 'Epoch 12/12')

In [78]:

history.model.predict(x_test)

#model.predict(x_test)

array([[1.7501545e-08, 3.1909576e-07, 7.4528231e-08, ..., 6.3826334e-08,
        9.9078527e-08, 2.3863764e-08],
       [1.8744319e-09, 3.0489575e-09, 1.0331556e-09, ..., 5.2834519e-09,
        1.0177007e-09, 3.9180690e-09],
       [1.2511714e-09, 4.7972337e-10, 4.5640047e-10, ..., 6.3720279e-10,
        1.0009170e-09, 5.1410716e-09],
       ...,
       [9.2204333e-09, 3.8032059e-08, 1.9962897e-08, ..., 4.6947026e-08,
        3.1969691e-08, 2.7271682e-08],
       [3.4061262e-10, 3.4958902e-10, 5.7367049e-09, ..., 2.8482179e-09,
        7.2771994e-10, 4.3123033e-11],
       [2.1282958e-07, 6.0935918e-07, 1.2954619e-07, ..., 3.8903593e-07,
        1.6377267e-07, 3.8244952e-08]], dtype=float32)

In [79]:
score = history.model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Test accuracy:', score[0])

7597/7597 [==============================] - 2s 293us/step
Test accuracy: 13.7228929594471


In [80]:
df_repar.head()

,tweet,compound,negative,neutral,positive,clean_text
0,He needs to shut up. He wants google to provid...,0.0000,0.000,1.000,0.000,needs shut wants google provide con content ti...
1,@BarackObama was a good lesson for Black Ameri...,-0.5095,0.189,0.735,0.076,barackobama good lesson black americans enemie...
2,Where are my reparations?? https://www.globalr...,0.0000,0.000,1.000,0.000,reparations https wwwglobalresearchca theirish...
3,But the 1 point that made the most sense which...,-0.8685,0.223,0.650,0.127,1 point made sense eventually led 2 reparation...
4,Let's talk reparations: https://youtu.be/qSF95...,0.0000,0.000,1.000,0.000,lets talk reparations https youtube qsf95mcgwp4


In [82]:
word = re.compile('[^\s]+', string= i)

for word in df_pingwords['ping pair']:
  print(word)

TypeError: ignored